In [145]:
from torch import nn
from torch.nn import functional as F
import torch
import torchvision
import torchvision.models as models
from torch.nn import Module
from torch import sigmoid
from pathlib import Path
import os

In [146]:
resnet34 = models.resnet34(pretrained=True)
print(resnet34)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [153]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, padding=1, kernel_size=3, stride=1, with_nonlinearity=True):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, padding=padding, kernel_size=kernel_size, stride=stride)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.with_nonlinearity = with_nonlinearity

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        if self.with_nonlinearity:
            x = self.relu(x)
        return x
    
class UpBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        
        self.upsample = nn.ConvTranspose2d(in_channels//2, in_channels//2, kernel_size=2, stride=2)

        self.conv_block_1 = ConvBlock(in_channels, out_channels)
        self.conv_block_2 = ConvBlock(out_channels, out_channels)

    def forward(self, up_x, down_x):
        x = self.upsample(up_x)
        
        diffY = down_x.size()[2] - x.size()[2]
        diffX = down_x.size()[3] - x.size()[3]
        
        x = F.pad(x, (diffX // 2, diffX - diffX//2,
                     diffY // 2, diffY - diffY//2))
        
        x = torch.cat([down_x, x], 1)
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        return x
    
class Bridge(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.bridge = nn.Sequential(
            ConvBlock(in_channels, out_channels),
            ConvBlock(out_channels, out_channels)
        )

    def forward(self, x):
        return self.bridge(x)

class ResNetUNet(nn.Module):
    DEPTH = 6

    def __init__(self, n_classes, basemodel):
        super().__init__()

        down_blocks = []
        up_blocks = []
      
        
        self.base_layers = list(basemodel.children())

        self.input_block = nn.Sequential(*self.base_layers[:3])
        self.input_pool = self.base_layers[3]
        down_blocks.append(self.base_layers[4])
        down_blocks.append(self.base_layers[5])
        down_blocks.append(self.base_layers[6])
        down_blocks.append(self.base_layers[7])
        
        self.down_blocks = nn.ModuleList(down_blocks)
        self.output_pool = nn.MaxPool2d(2, stride=2)
        #self.bridge = Bridge(2048, 2048)
        up_blocks.append(UpBlock(1024, 512))
        up_blocks.append(UpBlock(512, 256))
        up_blocks.append(UpBlock(256, 128))
        up_blocks.append(UpBlock(128, 64))
        up_blocks.append(UpBlock(64, 32))

        self.up_blocks = nn.ModuleList(up_blocks)
        
        self.out = nn.Conv2d(64, n_classes, kernel_size=1, stride=1)

    def forward(self, x):
        pre_pools = dict()
        pre_pools[f"layer_0"] = x
        x = self.input_block(x)
        pre_pools[f"layer_1"] = x
        x = self.input_pool(x)

        for i, block in enumerate(self.down_blocks, 2):
            x = block(x)
            if i == (ResNetUNet.DEPTH - 1):
                continue
            pre_pools[f"layer_{i}"] = x

        x = self.output_pool(x)

        for i, block in enumerate(self.up_blocks, 1):
            key = f"layer_{ResNetUNet.DEPTH - 1 - i}"
            x = block(x, pre_pools[key])
        x = self.out(x)
        del pre_pools
        return x    

In [154]:
def soft_jaccard(outputs, targets, weight=1):
    eps = 1e-15
    jaccard_target = (targets == 1).float()
    jaccard_output = sigmoid(outputs)

    intersection = (jaccard_output * jaccard_target).sum()
    union = jaccard_output.sum() + jaccard_target.sum()
    return intersection / (union - intersection + eps)

class LossBinary:

    def __init__(self, jaccard_weight=0):
        self.nll_loss = nn.BCEWithLogitsLoss()
        self.jaccard_weight = jaccard_weight

    def __call__(self, outputs, targets):
        loss = (1 - self.jaccard_weight) * self.nll_loss(outputs, targets)

        if self.jaccard_weight:
            loss += self.jaccard_weight * (1 - soft_jaccard(outputs, targets))
        return loss

class DiceLoss:
  
  def __init__(self, dice_weight=1):
                self.nll_loss = nn.BCELoss()
                self.dice_weight = dice_weight
  
  def __call__(self, outputs, targets):
    loss = self.nll_loss(outputs, targets)
    
    if self.dice_weight:
      eps = 1e-15
      dice_target = (targets == 1).float()
      dice_output = outputs
      intersection = (dice_output * dice_target).sum()
      union = dice_output.sum() + dice_target.sum() + eps
      loss -= torch.log(2 * intersection / union)
      
      return loss


In [155]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNetUNet(n_classes=1, basemodel=resnet34)
model = model.to(device)
print(model)

ResNetUNet(
  (input_block): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
  )
  (input_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (down_blocks): ModuleList(
    (0): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, a

)


In [156]:
from PIL import Image

import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.optim as optim
import glob


In [157]:
# store roadextractiondata in current working directory for this to work
dataset_dir = Path.cwd() / 'roadextractiondata'
train_dir = dataset_dir / 'train'
val_dir = dataset_dir / 'valid'

train_image_paths = glob.glob(str(train_dir / "*.jpg"))
train_mask_paths = glob.glob(str(train_dir / "*.png"))

val_image_paths = glob.glob(str(val_dir / "*.jpg"))
val_mask_paths = glob.glob(str(val_dir / "*.png"))

"""
train_image_paths = train_dir.glob('*.jpg')
train_mask_paths = train_dir.glob('*.png')
val_image_paths = val_dir.glob('*.jpg')
val_mask_paths = val_dir.glob('*.png')
"""

class CustomDataset(Dataset):

    def __init__(self, image_paths, target_paths, train=True):
      self.image_paths = image_paths
      self.target_paths = target_paths
      self.transforms = transforms.Compose([transforms.RandomResizedCrop(size = 448, scale = (0.6, 1.4)),
                                            transforms.RandomRotation(degrees = 30),
                                            transforms.ToTensor()])
    
#       self.transforms = transforms.ToTensor()
    
    
    
    def __getitem__(self, index):
         
        image = Image.open(self.image_paths[index])
        mask = Image.open(self.target_paths[index]).convert('1')
        t_image = transforms.ColorJitter(brightness=0.2,contrast=0.2,hue=0.02)(image)
        t_image = self.transforms(t_image)
#         t_image = self.transforms(image)
        t_mask = self.transforms(mask)

#         t_image = transforms.Compose([transforms.ToTensor()])(image)
#         t_mask = transforms.Compose([transforms.ToTensor()])(mask)

        
        return t_image, t_mask
    
    def __len__(self): 
        # return count of sample we have
        return sum(1 for x in self.image_paths)
    
train_dataset = CustomDataset(train_image_paths, train_mask_paths, train=True)
val_dataset = CustomDataset(val_image_paths, val_mask_paths, train=False)

print("Size of train dataset:", len(train_dataset))
print("Size of val dataset :", len(val_dataset))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=6)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=6)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNetUNet(n_classes=1, basemodel=resnet34)
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-04, weight_decay = 1e-04)
criterion = DiceLoss()

Size of train dataset: 6226
Size of val dataset : 1243


# Training

In [158]:
import datetime
print("Started at:", datetime.datetime.now())

for epoch in range(35):  # loop over the dataset multiple times (about 20k batches total)
#for epoch in range(1):  # loop over the dataset multiple times
    model.train()
    running_loss = 0.0
    
    for i, (image, mask) in enumerate(train_loader):
        # get the inputs
        inputs, mask = image.to(torch.cuda.current_device()), mask.to(torch.cuda.current_device())

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, mask)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 20 mini-batches
          print('[%d, %5d] loss: %.3f, accuracy: %.3f'  % (epoch + 1, i + 1, running_loss/20, soft_jaccard (outputs, mask)))
          running_loss = 0.0
          
        
        if i % 100 == 99:    # save every 100 mini-batches
          torch.save(model.state_dict(), 'classifier.pth')
        
#         if i % 20 == 19:
#           break

print('Saving')
torch.save(model.state_dict(), 'classifier.pth')

print('Finished Training')
print("Finished at:", datetime.datetime.now())

Started at: 2019-06-17 19:51:56.640929


RuntimeError: Given groups=1, weight of size 512 1024 3 3, expected input[6, 768, 28, 28] to have 1024 channels, but got 768 channels instead

In [57]:
import matplotlib.pyplot as plt
import numpy as np

# torch.save(model, 'classifier.pth')

# model = torch.load(model, 'classifier.pth')
# # checkpoint = torch.load('classifier.pth')
# # model.load_state_dict(checkpoint)



model = ResNetUNet(n_classes=1, basemodel=resnet34)
model = model.to(device)
model.load_state_dict(torch.load('classifier.pth'))
model.eval()



print("Done loading.")

model.eval()

running_loss = 0.0
  
with torch.no_grad():
      for i, (image, mask) in enumerate(val_loader):
        # get the inputs
        inputs, mask = image.to(torch.cuda.current_device()), mask.to(torch.cuda.current_device())

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, mask)
        
        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 20 mini-batches
#           print('[%d, %5d] loss: %.3f, accuracy: %.3f'  % (epoch + 1, i + 1, running_loss/20, soft_jaccard (outputs, mask)))
          print('[%d, %5d] loss: %.3f, accuracy: %.3f'  % (-1, i + 1, running_loss/20, soft_jaccard (outputs, mask)))
          running_loss = 0.0




print('Finished Eval')

Done loading.
Finished Eval


In [95]:
# Display validation results

import matplotlib.pyplot as plt
import numpy as np

# torch.save(model, 'classifier.pth')

# model = torch.load(model, 'classifier.pth')
# # checkpoint = torch.load('classifier.pth')
# # model.load_state_dict(checkpoint)




model = ResNetUNet(n_classes=1, basemodel=resnet34)
model = model.to(device)
model.load_state_dict(torch.load('classifier.pth'))
model.eval()



print("Done loading.")

model.eval()


iters = 10

with torch.no_grad():
  for i, (image, mask) in enumerate(val_loader):
    
    if i >= iters:
      break
      
#     if (i+1) not in [3,5,8,9,10]:
#       continue
    
    inputs, mask = image.to(torch.cuda.current_device()), mask.to(torch.cuda.current_device())
    outputs = model(inputs)
    loss = criterion(outputs, mask)
    
#     print('[%d, %5d] loss: %.3f, accuracy: %.3f'  % (epoch + 1, i + 1, loss, soft_jaccard (outputs, mask)))
    print('[%d, %5d] loss: %.3f, accuracy: %.3f'  % (-1, i + 1, loss, soft_jaccard (outputs, mask)))

    
    cmap = "hot"
    cmap = "nipy_spectral"
    cmap = "viridis" # default
    
    fig=plt.figure(figsize=(8, 8))
    columns = 3
    rows = 1
    
    fig.add_subplot(rows, columns, 1)
    inputplot = plt.imshow(inputs.cpu()[0][0], cmap=cmap)
    fig.add_subplot(rows, columns, 2)
    maskplot = plt.imshow(mask.cpu()[0][0], cmap=cmap)
    fig.add_subplot(rows, columns, 3)
    outplot = plt.imshow(outputs.cpu()[0][0], cmap=cmap)
    
    
    plt.show()

      

Done loading.
